# Learning Objectives

- Understand how to define and expose Python functions as tools using the Model Context Protocol (MCP) server (`FastMCP`).
- Learn how to configure and launch an MCP server using the `stdio` transport mechanism.
- Grasp how to connect a LangChain/LangGraph agent to an MCP server, dynamically load the available tools, and utilize them within an agent workflow (`stdio_client`, `ClientSession`, `load_mcp_tools`).

# Setup

In [ ]:
!pip install -q yfinance==0.2.56 \
                langchain==0.3.24 \
                langchain-openai==0.3.14 \
                langgraph==0.3.34 \
                langchain-mcp-adapters==0.0.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Business Scenario

Imagine a financial services company wants to build an internal AI assistant for its analysts. This assistant needs to answer questions about stocks, requiring access to real-time pricing and company information. Instead of embedding API keys or complex data-fetching logic directly into the agent (which might be running in a less secure environment or be harder to update), the company creates a dedicated MCP server.

This server securely wraps the necessary functions (like querying Yahoo Finance). The analyst interacts with the AI assistant (the client-side code), which securely communicates with the MCP tool server via stdio to fetch the required data, enabling the assistant to provide informed answers without directly handling the data source interaction itself. This pattern allows for easy updates to the tools (just update the server) and better separation of concerns.

# Implementation Roadmap

1. Goal: Create an LLM-based agent that can answer financial questions using real-time data from Yahoo Finance.
2. Challenge: LLMs lack direct, real-time access to external APIs like Yahoo Finance. We need a way to provide this capability.
3. Solution Approach: Use the Model Context Protocol (MCP) to bridge the gap.
    - Server (`yfserver.py`):
        - Define Tools: Create Python functions (`get_current_price`, `get_company_info`, `list_popular_tickers`) that use the yfinance library to fetch the required data.
        - Expose Tools via MCP: Use the `FastMCP` library and the `@mcp.tool()` decorator to expose these Python functions as callable tools over the MCP protocol. The decorator helps automatically define the tool's schema (inputs, outputs, description) based on type hints and docstrings.
        - Run the Server: Configure the MCP server to run and communicate using `stdio` (standard input/standard output). This is a simple transport mechanism suitable for local process communication.
    - Client (Main Notebook/Script):
        - Configure LLM: Set up the connection to the desired Large Language Model (e.g., ChatOpenAI).
        - Launch Server Process: Define how to start the `yfserver.py` script as a separate process (`StdioServerParameters`).
        - Establish MCP Connection: Use the `stdio_client` context manager to start the server process and get access to its stdin and stdout streams. Inside this, use the `ClientSession` context manager to establish an MCP session over these streams and perform the initial handshake (`session.initialize()`).
        - Discover Tools: Utilize the `langchain-mcp-adapters` library (`load_mcp_tools`) to automatically query the connected MCP server, discover the available tools, and convert them into LangChain-compatible Tool objects.
        - Create Agent: Instantiate a LangChain/LangGraph agent (like `create_react_agent`), providing it with the LLM and the dynamically loaded tools.
        - Execute Query: Pass the user's natural language query to the agent. The agent will use the LLM to plan steps, decide when to use a tool, invoke the tool (which triggers communication over MCP to the server process), receive the results, and generate a final response.

4. Rationale: This architecture decouples the agent's core reasoning logic (LLM) from the specific implementation of the tools. The MCP server acts as a secure and maintainable gateway to the external Yahoo Finance functionality. `stdio` is chosen for simplicity in this example, allowing the client script to directly manage the server process lifecycle. The `langchain-mcp-adapters` library simplifies the integration by handling the MCP communication details and tool conversion automatically.

# Server-side Implementation

In [ ]:
# -- SERVER SIDE --

%%writefile yfserver.py


import yfinance as yf

from typing import List, Dict, Any, Optional
from pydantic import BaseModel
from datetime import datetime

from mcp.server.fastmcp import FastMCP, Context

mcp = FastMCP("Yahoo Finance API")

class SectorFilter(BaseModel):
    sector: Optional[str] = None

@mcp.tool()
def get_current_price(ticker: str, ctx: Context = None) -> Dict[str, Any]:
    """
    Fetch the current price of a stock

    Args:
        ticker: Stock symbol (e.g., AAPL, MSFT)
    """
    if ctx:
        ctx.info(f"Fetching current price for {ticker}") # Server-side logging via MCP context

    try:
        ticker_obj = yf.Ticker(ticker)
        # Get the last closing price as a proxy for "current" price
        hist = ticker_obj.history(period="1d")
        if hist.empty:
            return {"error": "No data available"}

        price = hist['Close'].iloc[-1]
        return {
            "symbol": ticker,
            "price": round(price, 2),
            "currency": "USD",
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        return {"error": str(e)}

@mcp.tool()
def get_company_info(ticker: str, ctx: Context = None) -> Dict[str, Any]:
    """
    Get detailed company information

    Args:
        ticker: Stock symbol (e.g., AAPL, MSFT)
    """
    if ctx:
        ctx.info(f"Fetching company information for {ticker}")

    try:
        ticker_obj = yf.Ticker(ticker)
        info = ticker_obj.info

        # Extract relevant fields
        relevant_info = {
            "symbol": ticker,
            "name": info.get("shortName", "Unknown"),
            "longName": info.get("longName", "Unknown"),
            "sector": info.get("sector", "Unknown"),
            "industry": info.get("industry", "Unknown"),
            "website": info.get("website", "Unknown"),
            "marketCap": info.get("marketCap", None),
            "forwardPE": info.get("forwardPE", None),
            "dividendYield": info.get("dividendYield", None) if info.get("dividendYield") else None,
            "trailingEps": info.get("trailingEps", None),
            "description": info.get("longBusinessSummary", "No description available")
        }

        return relevant_info
    except Exception as e:
        return {"error": str(e)}

@mcp.tool()
def list_popular_tickers(filter: SectorFilter = SectorFilter()) -> Dict[str, Any]:
    """
    List popular stock tickers, optionally filtered by sector

    Args:
        filter: SectorFilter with optional sector name (Technology, Consumer, Financial, Healthcare, Energy)
    """

    # Hardcoded list for simplicity
    popular_tickers = {
        "Technology": ["AAPL", "MSFT", "GOOGL", "META", "NVDA"],
        "Consumer": ["AMZN", "WMT", "COST", "MCD", "NKE"],
        "Financial": ["JPM", "BAC", "V", "MA", "BRK-B"],
        "Healthcare": ["JNJ", "PFE", "UNH", "ABBV", "MRK"],
        "Energy": ["XOM", "CVX", "COP", "SLB", "EOG"]
    }

    sector = filter.sector

    if sector and sector in popular_tickers:
         # Return tickers for the specified sector
        return {
            "sector": sector,
            "tickers": [{
                "symbol": symbol,
                "uri": f"/tickers/{symbol}"
            } for symbol in popular_tickers[sector]]
        }
    elif sector:
        # Handle invalid sector input
        return {"error": f"Sector '{sector}' not found", "available_sectors": list(popular_tickers.keys())}
    else:
        # Return all popular tickers if no sector is specified
        result = {}
        for sector, tickers in popular_tickers.items():
            result[sector] = [{
                "symbol": symbol,
                "uri": f"/tickers/{symbol}"
            } for symbol in tickers]
        return result

if __name__ == "__main__":
    mcp.run(transport='stdio')

Writing yfserver.py


In the above code block:
- `%%writefile yfserver.py`: This Jupyter/Colab magic command saves the content of this cell into a file named `yfserver.py`. This file contains the server-side logic.
- `mcp = FastMCP("Yahoo Finance API")`: Initializes an MCP server instance, giving it a descriptive name.
- `class SectorFilter(BaseModel)`: Defines a Pydantic model to represent the expected structure for the filter argument in `list_popular_tickers`. It allows an optional sector string. Pydantic handles validation.
- `@mcp.tool()`: This decorator registers the decorated Python function (get_current_price, get_company_info, list_popular_tickers) as a tool accessible via the MCP protocol. MCP automatically uses the function's signature (name, arguments, type hints) and docstring to create a schema/description for the tool, which the client can discover.
- Tool Functions (`get_current_price`, etc.):
    - These functions implement the actual logic using the yfinance library.
    - They accept arguments as defined in their signature (e.g., `ticker: str`). `list_popular_tickers` uses the `SectorFilter` Pydantic model for its filter argument.
    - They include basic try...except blocks to handle potential errors during API calls and return an error dictionary if something goes wrong.
    - They return data structured as Python dictionaries, which MCP will serialize (likely to JSON) for transport.
    - The optional `ctx: Context` argument allows access to MCP server context, used here for logging (ctx.info).
- `if __name__ == "__main__": mcp.run(transport='stdio')`: This standard Python construct ensures that `mcp.run()` is called only when the script `yfserver.py` is executed directly. `mcp.run(transport='stdio')` starts the MCP server, making it listen for requests and send responses via the process's standard input and standard output streams.

# Client-side Implementation

## Setting up server access

In [ ]:
# -- Client Side --
import os
import asyncio
import subprocess

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import AzureChatOpenAI
from google.colab import userdata

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
azure_api_key = userdata.get('azure_api_key')
# Modify the Azure Endpoint and the API Versions as needed
azure_base_url = "https://oait3st.cognitiveservices.azure.com/"
azure_api_version = "2024-12-01-preview"

In [ ]:
llm = AzureChatOpenAI(
    azure_endpoint=azure_base_url,
    api_key=azure_api_key,
    api_version=azure_api_version,
    model='gpt-4o-mini',
    temperature=0
)

In [ ]:
server_params = StdioServerParameters(
    command="python",
    args=['yfserver.py']
)

In the above code block:
- `StdioServerParameters:` Configures how the client should launch the server process. It specifies the command (python) and the arguments (`yfserver.py`). This tells `stdio_client` how to start the server when needed.

## Main asynchronous function

In [ ]:
async def main(user_query):
    print(f"Starting MCP client for query: '{user_query}'")
    agent_response = None # Initialize response variable
    try:
        # Start the server process and establish stdio communication channels
        async with stdio_client(server_params, errlog=subprocess.PIPE) as (read, write):
            print("stdio_client connected. Initializing ClientSession...")
            try:
                # Establish an MCP session over the stdio channels
                async with ClientSession(read, write) as session:
                    await session.initialize() # Perform MCP handshake
                    print("ClientSession initialized. Loading tools...")
                    # Discover and load tools from the server via MCP
                    tools = await load_mcp_tools(session)
                    print(f"Tools loaded: {[tool.name for tool in tools]}")
                    print("Creating agent...")
                    # Create a ReAct agent with the LLM and loaded tools
                    agent = create_react_agent(llm, tools)
                    print("Invoking agent...")
                    # Prepare input for the agent
                    agent_input = {"messages": [("user", user_query)]}
                    agent_response = await agent.ainvoke(agent_input)
                    print("Agent finished.")
            except Exception as session_err:
                # Handle errors specific to the ClientSession context
                 print(f"\n--- ERROR within ClientSession context ---")
                 import traceback
                 traceback.print_exc()
                 agent_response = {"error": "ClientSession failed", "details": str(session_err)}

        # If we exit stdio_client normally, agent_response holds the result or None
        return agent_response
    except Exception as e:
        # Catch errors during stdio_client setup/teardown or other unexpected issues
        print(f"\n--- ERROR: An unexpected error occurred outside ClientSession ---")
        import traceback
        traceback.print_exc()
        print("---------------------------------------------------------------")
        return {"error": "Agent execution failed unexpectedly", "details": str(e)}


In the above code block:
- `async def main(user_query)`: Defines an asynchronous function that encapsulates the client logic for processing a single user query.
- `async with stdio_client(...)`: This asynchronous context manager:
    - Starts the server process using the `server_params` defined earlier (`python yfserver.py`).
    - Establishes communication by connecting to the server process's standard input and output.
    - Provides read and write asynchronous stream objects for communication.
    - `errlog=subprocess.PIPE` captures the server's standard error stream, useful for debugging server-side issues.
    - Ensures the server process is properly terminated when the context is exited.
- `async with ClientSession(read, write)`: Inside the `stdio_client` context, this creates an MCP `ClientSession` using the provided read and write streams. This session manages the MCP protocol communication.
- `await session.initialize()`: Performs the essential MCP handshake between the client and the server. During this phase, the client typically learns about the server's capabilities, including the available tools.
- `tools = await load_mcp_tools(session)`: This is the key integration point provided by langchain-mcp-adapters. It communicates with the initialized MCP session, discovers the tools defined on the server (like `get_current_price`), and automatically wraps them into LangChain-compatible Tool objects.
- `agent = create_react_agent(llm, tools)`: Creates a LangGraph agent using the create_react_agent prebuilt graph. It's configured with the llm and the list of tools obtained from the MCP server. The ReAct (Reasoning and Acting) agent framework allows the LLM to reason about which tool to use, invoke it, observe the result, and continue until it can answer the query.
- `agent_input = ...`: Formats the user query into the expected input structure for the LangGraph agent (a list of messages).
- `agent_response = await agent.ainvoke(agent_input)`: Executes the agent asynchronously with the user query. The agent will now potentially call the MCP tools (triggering communication through the ClientSession -> stdio_client -> server process) as part of its reasoning process.
- Error Handling: Includes `try...except` blocks to catch and report errors that might occur during the connection, session initialization, tool loading, or agent execution phases.

### Example 1

In [ ]:
user_query = "Apple company information?"

In [ ]:
response = asyncio.run(main(user_query))

Starting MCP client for query: 'Apple company information?'
stdio_client connected. Initializing ClientSession...
ClientSession initialized. Loading tools...
Tools loaded: ['get_current_price', 'get_company_info', 'list_popular_tickers']
Creating agent...
Invoking agent...
Agent finished.


In [ ]:
print(response['messages'][-1].content)

Here is the company information for Apple Inc. (Ticker: AAPL):

- **Name**: Apple Inc.
- **Sector**: Technology
- **Industry**: Consumer Electronics
- **Website**: [apple.com](https://www.apple.com)
- **Market Capitalization**: $3.19 trillion
- **Forward P/E Ratio**: 25.72
- **Dividend Yield**: 0.49%
- **Trailing EPS**: $6.41

**Description**: 
Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a range of products including:
- **iPhone**: A line of smartphones
- **Mac**: A line of personal computers
- **iPad**: A line of multi-purpose tablets
- **Wearables and Accessories**: Includes AirPods, Apple TV, Apple Watch, Beats products, and HomePod.

Apple also provides support services like AppleCare and cloud services, and operates platforms such as the App Store for applications and digital content. Additionally, it offers various subscription-based services including Apple Arcade, Apple Fitness+

### Example 2

In [ ]:
user_query = "Give me detailed company information about some popular stocks in the Technology sector"

In [ ]:
response = asyncio.run(main(user_query))

Starting MCP client for query: 'Give me detailed company information about some popular stocks in the Technology sector'
stdio_client connected. Initializing ClientSession...
ClientSession initialized. Loading tools...
Tools loaded: ['get_current_price', 'get_company_info', 'list_popular_tickers']
Creating agent...
Invoking agent...
Agent finished.


In [ ]:
print(response['messages'][-1].content)

Here is the detailed company information for some popular stocks in the Technology sector:

### 1. Apple Inc. (AAPL)
- **Name:** Apple Inc.
- **Sector:** Technology
- **Industry:** Consumer Electronics
- **Website:** [apple.com](https://www.apple.com)
- **Market Cap:** $3.19 trillion
- **Forward P/E:** 25.72
- **Dividend Yield:** 0.49%
- **Trailing EPS:** 6.41
- **Description:** Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers products like the iPhone, Mac, iPad, and various subscription-based services. Founded in 1976, it is headquartered in Cupertino, California.

---

### 2. Microsoft Corporation (MSFT)
- **Name:** Microsoft Corporation
- **Sector:** Technology
- **Industry:** Software - Infrastructure
- **Website:** [microsoft.com](https://www.microsoft.com)
- **Market Cap:** $3.80 trillion
- **Forward P/E:** 34.17
- **Dividend Yield:** 0.65%
- **Trailing EPS:** 12.93
- **Description:** 

<font size=6; color='blue'> **Happy Learning!** </font>
___